# How to define and execute basic forward-style workflows

* **Difficulty level**: easy
* **Time need to lean**: 10 minutes or less
* **Key points**:
  * A forward-style workflow consists of numerically numbered steps
  * Multiple workflows can be defined in a single SoS script or notebook
  * Optional input and output statements can be added to change how workflows are executed
  * The default input of a step if the output of its previous step  

## Simple workflows with numerically numbered steps

The workflows you have seen so far have numerically numbered steps. For example, this example from the [first tutorial](sos_in_notebook.html) have a single workflow `plot` with steps `plot_10` and `plot_20` and SoS will execute the steps in numeric ordering.

<div class="bs-callout bs-callout-primary" role="alert">
    <h4>Workflows with numerically numbered steps</h4>
    <ul>
        <li>Steps have the format of <code>name_index</code> (e.g. <code>step_10</code>) where <code>name</code> is the name of the workflow, and <code>index</code> is a numeric number</li>
        <li>Step indexes are usually not consecutive to allow easy insertion of new steps</li>
        <li>Both workflow <code>name</code> and <code>index</code> can be ignored. <code>10</code>, <code>20</code> etc are considered as steps of an unnamed workflow, <code>step</code> is considered as a one-step workflow</li>
        <li>Workflow can be executed by workflow name (e.g. <code>%run name</code> in Jupyter, <code>sos run name</code> from command line). A default workflow will be execute if only one workflow is defined, or a default workflow is defined</li>
    </ul>
</div>

In [1]:
%run

[global]
excel_file = 'data/DEG.xlsx'
csv_file = 'DEG.csv'
figure_file = 'output.pdf'

[plot_10]
run: expand=True
    xlsx2csv {excel_file} > {csv_file}

[plot_20]
R: expand=True
    data <- read.csv('{csv_file}')
    pdf('{figure_file}')
    plot(data$log2FoldChange, data$stat)
    dev.off()

INFO: Running plot_10 :

xlsx2csv data/DEG.xlsx > DEG.csv



INFO: Running plot_20 :

null device 
          1 


INFO: Workflow plot (ID=221e15922696f853) is executed successfully with 2 completed steps.

The workflow is executed by default with magic `%run` because only one workflow is defined in the script. You can also define multiple workflows and execute them by their names. For example, the following script defines two single-step workflows `convert` and `plot`. Because there is no default workflow, you will have to refer to them with their names:

```
%run convert
```
and
```
%run plot
```

In [2]:
%run convert

[global]
excel_file = 'data/DEG.xlsx'
csv_file = 'DEG.csv'
figure_file = 'output.pdf'

[convert]
run: expand=True
    xlsx2csv {excel_file} > {csv_file}

[plot]
R: expand=True
    data <- read.csv('{csv_file}')
    pdf('{figure_file}')
    plot(data$log2FoldChange, data$stat)
    dev.off()

INFO: Running convert :

xlsx2csv data/DEG.xlsx > DEG.csv



INFO: Workflow convert (ID=d799ecdba11b5554) is executed successfully with 1 completed step.

In [3]:
%run plot

[global]
excel_file = 'data/DEG.xlsx'
csv_file = 'DEG.csv'
figure_file = 'output.pdf'

[convert]
run: expand=True
    xlsx2csv {excel_file} > {csv_file}

[plot]
R: expand=True
    data <- read.csv('{csv_file}')
    pdf('{figure_file}')
    plot(data$log2FoldChange, data$stat)
    dev.off()

INFO: Running plot :

null device 
          1 


INFO: Workflow plot (ID=d9b491e59a96a0d0) is executed successfully with 1 completed step.

### Default input of steps

As shown in in [How to specify input and output files and process input files in groups](doc/user_guide/input_substeps.html), you can define `input` and `output` for each step. 

<div class="bs-callout bs-callout-primary" role="alert">
    <h4>Default <code>input</code> of numerically numbered workflows</h4>
    <p>The default input of a step in a numerically numbered workflow is the output of its previous step</p>
</div>

Therefore, in the following workflow, the `input` statement of `plot_20` can be ignored. 

In [6]:
%run

[global]
excel_file = 'data/DEG.xlsx'
csv_file = 'DEG.csv'
figure_file = 'output.pdf'

[plot_10]
input: excel_file
output: csv_file

run: expand=True
    xlsx2csv {_input} > {_output}

[plot_20]
# input: csv_file
output: figure_file

R: expand=True
    data <- read.csv('{_input}')
    pdf('{_output}')
    plot(data$log2FoldChange, data$stat)
    dev.off()

,plot,Workflow ID 1ca0da08be0c3ea0,Index #4,completed Ran for 0 sec


INFO: Running plot_10 :

xlsx2csv data/DEG.xlsx > DEG.csv



INFO: output: DEG.csv

INFO: Running plot_20 :

null device 
          1 


INFO: output: output.pdf

INFO: Workflow plot (ID=1ca0da08be0c3ea0) is executed successfully with 2 completed steps.

## Passing output of steps with substeps *

<div class="bs-callout bs-callout-warning" role="alert">
    <h4>Output of a step with substeps</h4>
    <p>If a step has multiple substeps, the step output consists of <code>_output</code> from each substep, which will be by default passed to the next step and create multiple substeps.</p>
</div>

Things can get a little bit complicated when a step has multiple substeps. As you can recall from [How to specify input and output files and process input files in groups](doc/user_guide/input_substeps.html), multiple substeps can be defined by input option `group_by`, each with its own `_input` and `_output`. When the output of such a step is inherited by another step, these `_output` will become the `_input` of the substeps.

For example, after running `fastqc` on the input fastq files, we would like to process the generated HTML file and check if the qualities are ok. We use the `beautifulsoup` Python module and find all the `<h2>` headers. Without going into the details of the use of `beautifulsoup` to parse HTML files, you should notice that 

* No `input` is defined for step `20` so it takes the output of step `10` as its input.
* The output of step `10` contains two groups, `data/S20_R1_fastqc.html` and `data/S20_R2_fastqc.html`, which becomes the input of two substeps of step `20`.
* The input of step `20` are processed one by one

In [24]:
%run 

[10]
input: 'data/S20_R1.fastq', 'data/S20_R2.fastq', group_by=1
output: f'{_input:n}_fastqc.html'

sh: expand=True
    fastqc {_input}
    
[20]

from bs4 import BeautifulSoup

with open(_input) as html:
    soup = BeautifulSoup(html)
    for h2 in soup.findAll('h2'):
        if h2.img:
            print(f"{_input:bn} {h2.text}: {h2.img['alt']}")

INFO: Running 10 :

INFO: 10 (index=0) is ignored due to saved signature

INFO: 10 (index=1) is ignored due to saved signature

INFO: output: data/S20_R1_fastqc.html data/S20_R2_fastqc.html in 2 groups

INFO: Running 20 :

S20_R1_fastqc Basic Statistics: [OK]
S20_R1_fastqc Per base sequence quality: [OK]
S20_R1_fastqc Per tile sequence quality: [OK]
S20_R1_fastqc Per sequence quality scores: [OK]
S20_R1_fastqc Per base sequence content: [FAIL]
S20_R1_fastqc Per sequence GC content: [FAIL]
S20_R1_fastqc Per base N content: [OK]
S20_R1_fastqc Sequence Length Distribution: [WARN]
S20_R1_fastqc Sequence Duplication Levels: [OK]
S20_R1_fastqc Overrepresented sequences: [FAIL]
S20_R1_fastqc Adapter Content: [OK]
S20_R2_fastqc Basic Statistics: [OK]
S20_R2_fastqc Per base sequence quality: [OK]
S20_R2_fastqc Per tile sequence quality: [OK]
S20_R2_fastqc Per sequence quality scores: [OK]
S20_R2_fastqc Per base sequence content: [FAIL]
S20_R2_fastqc Per sequence GC content: [FAIL]
S20_R2_fastqc Per base N content: [OK]
S20_R2_fastqc Sequence Length Distribution: [WARN]
S20_R2_fastqc Sequence Duplication Levels: [OK]
S20_R2_fastqc Overrepresented sequences: [FAIL]
S20_R2_fastqc Adapter Content: [OK]


INFO: Workflow default (ID=ee0072d3ce4e9adc) is executed successfully with 1 completed step, 2 completed substeps, 1 ignored step and 2 ignored substeps.

If you would like to re-group the default input, you can redefine the `input` explicitly, or apply option `group_by` to the default input:

In [27]:
%run -v0

[10]
input: 'data/S20_R1.fastq', 'data/S20_R2.fastq', group_by=1
output: f'{_input:n}_fastqc.html'

sh: expand=True
    fastqc {_input}
    
[20]
input: group_by='all'
print(f'Input of step 20 is {_input}')

Input of step 20 is data/S20_R1_fastqc.html data/S20_R2_fastqc.html


## Further reading
* [How to specify input and output files and process input files in groups](doc/user_guide/input_substeps.html)